In [19]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json, csv
# JSON 데이터 로드

data = []
with open ('./suneung_data.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    data = [{"id": row[0], "question": row[2]} for row in reader]

for d in data:
    print(d)


{'id': '0', 'question': 'Dear Rosydale City Marathon Racers, We are really grateful to all of you who have signed up for the 10th Rosydale City Marathon that was scheduled for this coming Saturday at 10 a.m. Unfortunately, as you may already know, the weather forecast says that there is going to be a downpour throughout the race day. We truly hoped that the race would go smoothly. However, it is likely that the heavy rain will make the roads too slippery and dangerous for the racers to run safely. As a result, we have decided to cancel the race. We hope you understand and we promise to hold another race in the near future. Sincerely, Martha Kingsley Race Manager'}
{'id': '1', 'question': 'It was Valentine’s Day on Friday and Peter was certain that his wife, Amy, was going to love his surprise. Peter had spent a long time searching online for an event that would be a new way to spend time with Amy. He had finally found the perfect thing for her. She often told him that she liked to go t

In [16]:
# 문제 본문 추출 및 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = [item["question"] for item in data]
ids = [item["id"] for item in data]  # id 리스트 생성
embeddings = model.encode(texts)

# FAISS 인덱스 생성 및 벡터 추가
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

# FAISS 인덱스 저장
faiss.write_index(index, 'faiss_index.index')

# id 리스트 저장 (FAISS와 관계형 DB 매핑용)
with open('ids.json', 'w') as f:
    json.dump(ids, f)


In [20]:
# FAISS 인덱스 및 ID 매핑 로드
index = faiss.read_index('faiss_index.index')
with open('ids.json', 'r') as f:
    ids = json.load(f)

# 사용자 입력 쿼리 처리
query = "The ability to understand emotions"  # 사용자 입력 질문
query_vector = model.encode([query]).astype('float32')

# FAISS 검색 수행
distances, indices = index.search(query_vector, k=1)  # 가장 유사한 질문 1개 검색

# 검색된 id 가져오기 (FAISS 결과 → id 매핑) 4
matched_id = ids[indices[0][0]]
print(f"검색된 문제 ID: {matched_id}")


검색된 문제 ID: 4
